## 1. Imports

In [1]:
import pandas as pd
import numpy as np
import yaml
import copy
import torch
import json
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sqlalchemy import create_engine
from pytorch_datasets import SentimentAnalysisDataset, DatasetType

In [2]:
model_name = "ProsusAI/finbert"

## 2. Database configuration & Model config

In [3]:
# Reading form config.yaml"
with open("../../config.yaml", "r") as yamlconfig:
    config = yaml.load(yamlconfig, Loader=yaml.FullLoader)

# Create postgres string with db-config
postgres_username = config["db_config"]["postgres_username"]
postgres_password = config["db_config"]["postgres_password"]
postgres_address = config["db_config"]["postgres_address"]
postgres_port = config["db_config"]["postgres_port"]
postgres_dbname = config["db_config"]["postgres_dbname"]

postgres_str = f"postgresql://{postgres_username}:{postgres_password}@{postgres_address}:{postgres_port}/{postgres_dbname}"

# create db connection with sqlalchemy
cnx = create_engine(postgres_str)

In [4]:
# Load json file with hyperparams of each model
with open('hyperparams.json') as file:
    hyper_params = json.load(file)

In [5]:
# Set up Hyper parameters for model training
LR: float = hyper_params[model_name]["lr"]
OPTIMIZER: str = hyper_params[model_name]["lr"]
EPOCHS: int = hyper_params[model_name]["epochs"]
BATCH_SIZE: int = hyper_params[model_name]["batch_size"]
DROPOUT: float = hyper_params[model_name]["dropout"]

## 3. Dataframe preperations

In [6]:
df = pd.read_excel("../../data/external/annotations_wsb.xlsx", sheet_name="annotations_scale")
df.dropna(inplace=True)
df.drop(columns=["ben", "dliden", "doss", "jmo", "jmo2", "id"], inplace=True)
df.rename(columns={"Own annotation": "label"}, inplace=True)
#dummies = pd.get_dummies(df['label'])
#df = pd.merge(
#    left=df,
#    right=dummies,
#    left_index=True,
#    right_index=True,
#)
#df.rename(columns={0.0: "Negative", 1.0: "Neutral", 2.0: "Positive"}, inplace=True)

In [7]:
df

,post,label
0,"\n \nGentlemen, start your boners:\n\nBlackBe...",2.0
1,#It has come to my attention that $PLTR Nation...,2.0
2,#SNDL and TLRY for smooking weed on the moon 🚬...,2.0
3,$63k in on WKHS calls,2.0
4,$CLF and $MT,1.0
...,...,...
178,Woke up on the floor got up and I looked perfe...,2.0
179,"Yeah, I was going to sell my UWMC for a L, but...",0.0
180,"Yep, it’s cheap right now and we could see it ...",2.0
181,You didn’t know what a hedge fund was before G...,1.0


In [8]:
df["label"].value_counts(normalize=True
                         )

2.0    0.519126
1.0    0.295082
0.0    0.185792
Name: label, dtype: float64

## 4. Model Loading

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## 3. Building Pytorch Dataset

In [11]:
# Declare generic sentiment analysis dataset without split
sentiment_analysis_dataset = SentimentAnalysisDataset(
    df = df,
    tokenizer = tokenizer
)

In [12]:
# Declare train and test dataset
train_dataset = copy.deepcopy(sentiment_analysis_dataset).set_fold(DatasetType.TRAIN)
test_dataset = copy.deepcopy(sentiment_analysis_dataset).set_fold(DatasetType.TEST)

In [13]:
# Setup train and test Data loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                num_workers=1,
                                                drop_last=False # maybe change in future
                                                )

test_data_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=test_dataset.__len__(),
                                              shuffle=False,
                                              num_workers=1,
                                              )

In [14]:
train_dataset.__len__()

146

In [15]:
test_dataset.__len__()

37

In [16]:
# Check if train data and test data have correct batch and tensor sizes
"""print('TRAINING DATA:')
for dictionary in train_data_loader:
    print(dictionary)
    break"""

print(' ')
print('TESTING DATA:')
for dictionary in test_data_loader:
    print(dictionary["labels"].size())
    #break

 
TESTING DATA:
torch.Size([37, 3])


## 6. Creating custom Model

In [17]:
# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [18]:
# Set up custom model
class SentimentAnalysisModel(torch.nn.Module):
    def __init__(self, pretrained_model):
        super(SentimentAnalysisModel, self).__init__()
        self.pretrained_model = pretrained_model
        #self.dropout = torch.nn.Dropout(0.3)
        #self.linear = torch.nn.Linear(768, 6)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.pretrained_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        #output_dropout = self.dropout(output.pooler_output)
        #output = self.linear(output_dropout)
        return output

model = SentimentAnalysisModel(model)
model.to(device)

SentimentAnalysisModel(
  (pretrained_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_fe

In [31]:
testbatch = next(iter(train_data_loader))
testbatch["attention_mask"]

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [32]:
outputs = model.forward(testbatch["input_ids"], testbatch["attention_mask"], testbatch["token_type_ids"])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [ ]:
outputs.logits

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
tokenizer_output = tokenizer("Stocks rallied and the British pound gained.")
tokenizer_output

In [55]:
tokenizer_output = tokenizer.encode_plus(
    "Stocks rallied and the British pound gained.",
    add_special_tokens=True,
    max_length=512,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

In [61]:
tokenizer_output["token_type_ids"].flatten()

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [71]:
with torch.no_grad():
    output = model(tokenizer_output["input_ids"], attention_mask = tokenizer_output["attention_mask"], token_type_ids = tokenizer_output["token_type_ids"]).logits
    print(output)

tensor([[ 1.6898, -1.5706, -0.9036]])


In [69]:
from torch.nn.functional import softmax

In [72]:
softmax(output, dim=1)

tensor([[0.8984, 0.0345, 0.0672]])

## 7. Model training

In [19]:
for epoch in tqdm(range(1, EPOCHS + 1)):
    print(epoch)

    train_loss = 0
    valid_loss = 0

    for batch_idx, data in enumerate(train_data_loader):
        #print(batch_idx)
        #print(data)

        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['labels'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        print(output)
        break

    break

  0%|          | 0/6 [00:00<?, ?it/s]

1


  0%|          | 0/6 [00:02<?, ?it/s]


NameError: name 'output' is not defined

In [20]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.8941, -2.3504,  1.4936],
        [ 0.0124, -1.3215,  1.3675],
        [ 0.1558, -1.8437,  1.9891],
        [ 1.3918, -2.2401,  0.5130],
        [-1.1547, -0.4517,  2.1827],
        [-0.3151, -1.5200,  2.0887],
        [-0.3807, -1.4364,  2.0338],
        [-0.8494, -0.7969,  2.3599],
        [ 0.4437, -2.5427,  2.0701],
        [-0.5694, -1.0252,  2.2709],
        [ 1.1075, -2.5437,  1.4357],
        [ 0.1128, -1.8091,  2.0302],
        [ 0.4829, -2.0726,  1.8711],
        [-0.7075, -0.8917,  2.3569],
        [-1.4921,  1.0854,  1.4847],
        [ 0.5051, -2.2887,  1.9963]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [21]:
# reihenfolge der target klassen anpassen

In [35]:
loss = torch.nn.BCEWithLogitsLoss()
input = torch.FloatTensor([0,1,0])
print(input)
target = torch.FloatTensor([1,0,0])
print(target)
output = loss(input, target, dim=1)
print(output)

tensor([0., 1., 0.])
tensor([1., 0., 0.])


TypeError: forward() got an unexpected keyword argument 'dim'

## WANDB test

In [26]:
import wandb

wandb.init(project="test-project", entity="hda_sis")

wandb: Currently logged in as: jan_burger (hda_sis). Use `wandb login --relogin` to force relogin


In [27]:
wandb.config = {
    "learning_rate": 0.001,
    "epochs": 100,
    "batch_size": 128
}

In [29]:
wandb.log({"loss": 2.5})